# Hiding the information inside the image using Steganography

In [2]:
!pip install pillow numpy matplotlib

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.8.4 requires typer<1.0.0,>=0.3.0, which is not installed.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.


In [42]:
#import required libraries
from PIL import Image
import numpy as np

In [44]:
# STEP 1: Convert text to binary
def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

In [46]:
# STEP 2: Convert binary back to text
def binary_to_text(binary_data):
    chars = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    return ''.join(chr(int(char, 2)) for char in chars if len(char) == 8)

In [54]:
# STEP 3: Encode message in image
def encode_image(image_path, secret_message, output_path):
    img = Image.open(image_path)
    img = img.convert("RGB")
    data = np.array(img)

    binary_secret = text_to_binary(secret_message) + '1111111111111110'  # End marker
    flat_data = data.flatten().astype(np.uint16)  # Use uint16 to avoid overflow

    if len(binary_secret) > len(flat_data):
        raise ValueError("Message is too long to hide in this image!")

    for i in range(len(binary_secret)):
        flat_data[i] = (flat_data[i] & 0xFFFE) | int(binary_secret[i])  # Clear LSB and set new

    new_data = flat_data.astype(np.uint8).reshape(data.shape)  # Convert back to uint8
    encoded_img = Image.fromarray(new_data)
    encoded_img.save(output_path)

    print(f"✅ Message hidden successfully in: {output_path}")

In [56]:
# Encode using the uploaded image path
encode_image(
    image_path= "C:\\Users\\saipr\\Downloads\\nature_image.png", # Uploaded image
    secret_message = ' Hello Ammu, this is secret!',          # Message to hide
    output_path= "C:\\Users\\saipr\\Downloads\\nature1_image.png"        # Output image
)

✅ Message hidden successfully in: C:\Users\saipr\Downloads\nature1_image.png


In [58]:
def decode_image(stego_image_path):
    img = Image.open(stego_image_path)
    img = img.convert("RGB")
    data = np.array(img).flatten()

    binary_data = ''
    for value in data:
        binary_data += str(value & 1)

    eof = binary_data.find('1111111111111110')
    if eof != -1:
        binary_data = binary_data[:eof]

    return binary_to_text(binary_data)

In [60]:
# Decode and print message
hidden_message = decode_image("C:\\Users\\saipr\\Downloads\\nature1_image.png")
print("🔓 Hidden Message:", hidden_message)

🔓 Hidden Message:  Hello Ammu, this is secret!
